# 9.5 RNN을 이용한 텍스트 생성(Text Generation using RNN)
___
이번 챕터에서는 다 대 일(many-to-one) 구조의 RNN을 사용하여 문맥을 반영해서 텍스트를 생성하는 모델을 만들어봅시다

### 1. RNN을 이용하여 텍스트 생성하기
예를 들어서 '경마장에 있는 말이 뛰고 있다'와 '그의 말이 법이다'와 '가는 말이 고와야 오는 말이 곱다'라는 세 가지 문장이 있다고 해봅시다. 모델이 문맥을 학습할 수 있도록 전체 문장의 앞의 단어들을 전부 고려하여 학습하도록 데이터를 재구성한다면 아래와 같이 총 11개의 샘플이 구성됩니다.

samples	|X	|y
--- | --- | ---
1	|경마장에	|있는
2	|경마장에 있는	|말이
3	|경마장에 있는 말이	|뛰고
4	|경마장에 있는 말이 뛰고	|있다
5	|그의	|말이
6	|그의 말이	|법이다
7	|가는	|말이
8	|가는 말이	|고와야
9	|가는 말이 고와야	|오는
10	|가는 말이 고와야 오는	|말이
11	|가는 말이 고와야 오는 |말이

### 1) 데이터에 대한 이해와 전처리

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

text="""경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

In [4]:
t = Tokenizer()
t.fit_on_texts([text])
vocab_size = len(t.word_index)+1

print('Size of Vocab Set : %d' % vocab_size)
print(t.word_index)

Size of Vocab Set : 12
{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [25]:
sequences = list()

for line in text.split('\n'):
    encoded = t.texts_to_sequences([line])[0]
    if len(encoded)>1: print('encoded : ', encoded)
    
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
        
print('\nsample count to use study : %d' % len(sequences))        
print('\n', sequences)    

encoded :  [2, 3, 1, 4, 5]
encoded :  [6, 1, 7]
encoded :  [8, 1, 9, 10, 1, 11]

sample count to use study : 11

 [[2, 3], [2, 3, 1], [2, 3, 1, 4], [2, 3, 1, 4, 5], [6, 1], [6, 1, 7], [8, 1], [8, 1, 9], [8, 1, 9, 10], [8, 1, 9, 10, 1], [8, 1, 9, 10, 1, 11]]


위의 데이터는 아직 레이블로 사용될 단어를 분리하지 않은 훈련 데이터입니다. [2, 3]은 [경마장에, 있는]에 해당되며 [2, 3, 1]은 [경마장에, 있는, 말이]에 해당됩니다. 전체 훈련 데이터에 대해서 맨 우측에 있는 단어에 대해서만 레이블로 분리해야 합니다.

우선 전체 샘플에 대해서 길이를 일치시켜 줍니다. 가장 긴 샘플의 길이를 기준으로 합니다. 현재 육안으로 봤을 때, 길이가 가장 긴 샘플은 [8, 1, 9, 10, 1, 11]이고 길이는 6입니다. 이를 코드로는 다음과 같이 구할 수 있습니다.

In [26]:
max_len = max(len(l) for l in sequences)
print(' Max length of sample : %d' % max_len)

 Max length of sample : 6


전체 훈련 데이터에서 가장 긴 샘플의 길이가 6임을 확인하였습니다. 이제 전체 샘플의 길이를 6으로 패딩합니다.

In [28]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


길이가 6보다 짧은 모든 샘플에 대해서 앞에 0을 채워서 모든 샘플의 길이를 6으로 바꿨습니다. 이제 각 샘플의 마지막 단어를 레이블로 분리합시다. 레이블의 분리는 Numpy를 이용해서 가능합니다.

In [29]:
sequences = np.array(sequences)
X = sequences[:, :-1]
y = sequences[:, -1]
print(X)
print(y)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]
[ 3  1  4  5  1  7  1  9 10  1 11]


레이블이 분리되었습니다. 이제 RNN 모델에 훈련 데이터를 훈련 시키기 전에 레이블에 대해서 원-핫 인코딩을 수행합니다.

In [31]:
y = to_categorical(y, num_classes=vocab_size)
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


### 2) 모델 설계하기
이제 RNN 모델에 데이터를 훈련시킵니다.

In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [37]:
# 각 단어의 임베딩 벡터는 10차원을 가지고, 
# 32의 은닉 상태 크기를 가지는 바닐라 RNN을 사용합니다..
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X,y, epochs=200, verbose=2)

Epoch 1/200
11/11 - 0s - loss: 2.4821 - accuracy: 0.0909
Epoch 2/200
11/11 - 0s - loss: 2.4692 - accuracy: 0.1818
Epoch 3/200
11/11 - 0s - loss: 2.4564 - accuracy: 0.3636
Epoch 4/200
11/11 - 0s - loss: 2.4436 - accuracy: 0.4545
Epoch 5/200
11/11 - 0s - loss: 2.4306 - accuracy: 0.2727
Epoch 6/200
11/11 - 0s - loss: 2.4174 - accuracy: 0.3636
Epoch 7/200
11/11 - 0s - loss: 2.4039 - accuracy: 0.3636
Epoch 8/200
11/11 - 0s - loss: 2.3899 - accuracy: 0.3636
Epoch 9/200
11/11 - 0s - loss: 2.3753 - accuracy: 0.3636
Epoch 10/200
11/11 - 0s - loss: 2.3601 - accuracy: 0.3636
Epoch 11/200
11/11 - 0s - loss: 2.3443 - accuracy: 0.3636
Epoch 12/200
11/11 - 0s - loss: 2.3278 - accuracy: 0.3636
Epoch 13/200
11/11 - 0s - loss: 2.3106 - accuracy: 0.3636
Epoch 14/200
11/11 - 0s - loss: 2.2925 - accuracy: 0.3636
Epoch 15/200
11/11 - 0s - loss: 2.2737 - accuracy: 0.3636
Epoch 16/200
11/11 - 0s - loss: 2.2542 - accuracy: 0.3636
Epoch 17/200
11/11 - 0s - loss: 2.2339 - accuracy: 0.3636
Epoch 18/200
11/11 - 0s

Epoch 142/200
11/11 - 0s - loss: 0.2393 - accuracy: 1.0000
Epoch 143/200
11/11 - 0s - loss: 0.2348 - accuracy: 1.0000
Epoch 144/200
11/11 - 0s - loss: 0.2303 - accuracy: 1.0000
Epoch 145/200
11/11 - 0s - loss: 0.2259 - accuracy: 1.0000
Epoch 146/200
11/11 - 0s - loss: 0.2216 - accuracy: 1.0000
Epoch 147/200
11/11 - 0s - loss: 0.2174 - accuracy: 1.0000
Epoch 148/200
11/11 - 0s - loss: 0.2133 - accuracy: 1.0000
Epoch 149/200
11/11 - 0s - loss: 0.2093 - accuracy: 1.0000
Epoch 150/200
11/11 - 0s - loss: 0.2053 - accuracy: 1.0000
Epoch 151/200
11/11 - 0s - loss: 0.2015 - accuracy: 1.0000
Epoch 152/200
11/11 - 0s - loss: 0.1977 - accuracy: 1.0000
Epoch 153/200
11/11 - 0s - loss: 0.1940 - accuracy: 1.0000
Epoch 154/200
11/11 - 0s - loss: 0.1903 - accuracy: 1.0000
Epoch 155/200
11/11 - 0s - loss: 0.1868 - accuracy: 1.0000
Epoch 156/200
11/11 - 0s - loss: 0.1833 - accuracy: 1.0000
Epoch 157/200
11/11 - 0s - loss: 0.1799 - accuracy: 1.0000
Epoch 158/200
11/11 - 0s - loss: 0.1765 - accuracy: 1.00

모델이 정확하게 예측하고 있는지 문장을 생성하는 함수를 만들어서 출력해봅시다.



In [38]:
def sentence_generation(model, t, current_word, n):
    ''' 
    model : 모델
    t : Tokenizer
    current_word : 현재 단어
    n: 반복할 횟수
    '''
    init_word = current_word
    sentence =''
    
    for _ in range(n):
        encoded = t.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=5, padding='pre')
        result = model.predict_classes(encoded, verbose=0)
        
        for word, index in t.word_index.items():
            if index == result:
                break
        current_word = current_word + ' ' + word
        sentence = sentence + ' ' + word
        
    sentencs = init_word + sentence
    return sentence

이제 입력된 단어로부터 다음 단어를 예측해서 문장을 생성하는 함수를 만들었습니다.

In [40]:
print(sentence_generation(model, t, '경마장에',4))

 있는 말이 뛰고 있다


In [41]:
print(sentence_generation(model, t, '그의',2))

 말이 법이다


In [43]:
print(sentence_generation(model, t, '가는',5))

 말이 고와야 오는 말이 곱다


이제 앞의 문맥을 기준으로 '말이' 라는 단어 다음에 나올 단어를 기존의 훈련 데이터와 일치하게 예측함을 보여줍니다. 이 모델은 충분한 훈련 데이터를 갖고 있지 못하므로 위에서 문장의 길이에 맞게 적절하게 예측해야하는 횟수 4, 2, 5를 각각 인자값으로 주었습니다. 이 이상의 숫자를 주면 기계는 '있다', '법이다', '곱다' 다음에 나오는 단어가 무엇인지 배운 적이 없으므로 임의 예측을 합니다. 이번에는 더 많은 훈련 데이터를 가지고 실습해봅시다.

### 2. LSTM을 이용하여 텍스트 생성하기
이번에는 LSTM을 통해 보다 많은 데이터로 텍스트를 생성해보겠습니다. 본질적으로 앞에서 한 것과 동일한 실습입니다.

##### 1) 데이터에 대한 이해와 전처리
사용할 데이터는 뉴욕 타임즈 기사의 제목입니다. 아래의 링크에서 ArticlesApril2018.csv 데이터를 다운로드 합니다.

파일 다운로드 링크 : https://www.kaggle.com/aashita/nyt-comments

In [47]:
import pandas as pd
import numpy as np
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [51]:
df = pd.read_csv('../../data/kaggle/ArticlesApril2018.csv')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleaders’ Settlement Offer: ...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"“I understand that they could meet with us, pa...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,What’s it like to eat at the second incarnatio...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Toronto’s Ri...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


In [53]:
print('Count of Column :', len(df.columns))
print(df.columns)

Count of Column : 15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [55]:
df['headline'].isnull().values.any()

False

In [56]:
headline = []
headline.extend(list(df.headline.values))
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

그런데 4번째, 5번째 샘플에 Unknown 값이 들어가있습니다. headline 전체에 걸쳐서 Unknown 값을 가진 샘플이 있을 것으로 추정됩니다. 비록 Null 값은 아니지만 지금 하고자 하는 실습에 도움이 되지 않는 노이즈 데이터이므로 제거해줄 필요가 있습니다. 제거하기 전에 현재 샘플의 개수를 확인해보고 제거 전, 후의 샘플의 개수를 비교해보겠습니다.

In [57]:
print('Total count of sample : {}'.format(len(headline)))

Total count of sample : 1324


In [58]:
headline = [ hn for hn in headline if hn !='Unknown']

In [59]:
print('Total count of sample after removing noise data : {}'.format(len(headline)))

Total count of sample after removing noise data : 1214


In [60]:
headline[:5]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

기존에 4번째, 5번째 샘플에서는 Unknown 값이 있었는데 현재는 제거가 된 것을 확인하였습니다. 이제 데이터 전처리를 수행합니다. 여기서 선택한 전처리는 구두점 제거와 단어의 소문자화입니다. 전처리를 수행하고, 다시 샘플 5개를 출력합니다.

In [61]:
def repreprocessing(s):
    s = s.encode('utf8').decode('ascii', 'ignore')
    return ''.join(c for c in s if c not in punctuation).lower()

text = [ repreprocessing(x) for x in headline]
text[:5]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression']

기존의 출력과 비교하면 모든 단어들이 소문자화되었으며 N.F.L.이나 Cheerleaders’ 등과 같이 기존에 구두점이 붙어있던 단어들에서 구두점이 제거되었습니다. 이제 단어 집합(vocabulary)을 만들고 크기를 확인합니다.

In [62]:
t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) + 1
print('Size of Vocab set : %d'%vocab_size)

Size of Vocab set : 3494


총 3,494개의 단어가 존재합니다. 이제 정수 인코딩과 동시에 하나의 문장을 여러 줄로 분해하여 훈련 데이터를 구성합니다.

In [64]:
sequences = list()

for line in text:
    encoded = t.texts_to_sequences([line])[0]
    
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)
        
sequences[:11]        

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

이해를 돕기 위해 출력 결과에 주석을 추가하였습니다. 왜 하나의 문장을 저렇게 나눌까요? 예를 들어 '경마장에 있는 말이 뛰고 있다' 라는 문장 하나가 있을 때, 최종적으로 원하는 훈련 데이터의 형태는 다음과 같습니다. 하나의 단어를 예측하기 위해 이전에 등장한 단어들을 모두 참고하는 것입니다.

위의 sequences는 모든 문장을 각 단어가 각 시점(time step)마다 하나씩 추가적으로 등장하는 형태로 만들기는 했지만, 아직 예측할 단어에 해당되는 레이블을 분리하는 작업까지는 수행하지 않은 상태입니다. 어떤 정수가 어떤 단어를 의미하는지 알아보기 위해 인덱스로부터 단어를 찾는 index_to_word를 만듭니다.

In [65]:
index_to_word ={}
for key, value in t.word_index.items():
    index_to_word[value] = key

print('Top 582 frequency words : {}'.format(index_to_word[582]))    

Top 582 frequency words : offer


582이라는 인덱스를 가진 단어는 본래 offer라는 단어였습니다. 원한다면 다른 숫자로도 시도해보세요. 이제 y데이터를 분리하기 전에 전체 샘플의 길이를 동일하게 만드는 패딩 작업을 수행합니다. 패딩 작업을 수행하기 전에 가장 긴 샘플의 길이를 확인합니다.

In [66]:
max_len = max(len(s) for s in sequences)
print('Max length of samples : {}'.format(max_len))


Max length of samples : 24


In [67]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0   99  269]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0   99  269  371]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0   99  269  371 1115]]


padding='pre'를 설정하여 샘플의 길이가 24보다 짧은 경우에 앞에 0으로 패딩되었습니다. 이제 맨 우측 단어만 레이블로 분리합니다.

In [68]:
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

print(X[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  99]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0  99 269]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0  99 269 371]]


훈련 데이터 X에서 3개의 샘플만 출력해보았는데, 맨 우측에 있던 정수값 269, 371, 1115가 사라진 것을 볼 수 있습니다. 뿐만 아니라, 각 샘플의 길이가 24에서 23으로 줄었습니다.

In [69]:
y = to_categorical(y, num_classes = vocab_size)

레이블 데이터 y에 대해서 원-핫 인코딩을 수행하였습니다. 이제 모델을 설계합니다.



##### 2) 모델 설계하기

In [70]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM

In [73]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X,y, epochs=200, verbose=2)

Epoch 1/200
7803/7803 - 10s - loss: 7.6497 - accuracy: 0.0240
Epoch 2/200
7803/7803 - 9s - loss: 7.1179 - accuracy: 0.0309
Epoch 3/200
7803/7803 - 9s - loss: 6.9792 - accuracy: 0.0345
Epoch 4/200
7803/7803 - 8s - loss: 6.8551 - accuracy: 0.0417
Epoch 5/200
7803/7803 - 9s - loss: 6.7059 - accuracy: 0.0460
Epoch 6/200
7803/7803 - 9s - loss: 6.5409 - accuracy: 0.0455
Epoch 7/200
7803/7803 - 9s - loss: 6.3606 - accuracy: 0.0520
Epoch 8/200
7803/7803 - 8s - loss: 6.1699 - accuracy: 0.0595
Epoch 9/200
7803/7803 - 8s - loss: 5.9851 - accuracy: 0.0628
Epoch 10/200
7803/7803 - 8s - loss: 5.7989 - accuracy: 0.0624
Epoch 11/200
7803/7803 - 8s - loss: 5.6194 - accuracy: 0.0695
Epoch 12/200
7803/7803 - 9s - loss: 5.4481 - accuracy: 0.0719
Epoch 13/200
7803/7803 - 9s - loss: 5.2892 - accuracy: 0.0793
Epoch 14/200
7803/7803 - 11s - loss: 5.1403 - accuracy: 0.0852
Epoch 15/200
7803/7803 - 8s - loss: 4.9925 - accuracy: 0.0946
Epoch 16/200
7803/7803 - 8s - loss: 4.8521 - accuracy: 0.1041
Epoch 17/200
78

Epoch 132/200
7803/7803 - 19s - loss: 0.3678 - accuracy: 0.9132
Epoch 133/200
7803/7803 - 16s - loss: 0.3633 - accuracy: 0.9141
Epoch 134/200
7803/7803 - 14s - loss: 0.3615 - accuracy: 0.9131
Epoch 135/200
7803/7803 - 15s - loss: 0.3556 - accuracy: 0.9138
Epoch 136/200
7803/7803 - 15s - loss: 0.3478 - accuracy: 0.9148
Epoch 137/200
7803/7803 - 16s - loss: 0.3446 - accuracy: 0.9153
Epoch 138/200
7803/7803 - 23s - loss: 0.3390 - accuracy: 0.9148
Epoch 139/200
7803/7803 - 23s - loss: 0.3371 - accuracy: 0.9146
Epoch 140/200
7803/7803 - 21s - loss: 0.3337 - accuracy: 0.9138
Epoch 141/200
7803/7803 - 16s - loss: 0.3309 - accuracy: 0.9135
Epoch 142/200
7803/7803 - 19s - loss: 0.3275 - accuracy: 0.9159
Epoch 143/200
7803/7803 - 17s - loss: 0.3229 - accuracy: 0.9149
Epoch 144/200
7803/7803 - 17s - loss: 0.3191 - accuracy: 0.9162
Epoch 145/200
7803/7803 - 16s - loss: 0.3178 - accuracy: 0.9153
Epoch 146/200
7803/7803 - 16s - loss: 0.3134 - accuracy: 0.9176
Epoch 147/200
7803/7803 - 16s - loss: 0.

In [75]:
def sentence_generation(model, t, current_word, n):
    init_word = current_word
    sentence =''
    for _ in range(n):
        encoded = t.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=23, padding='pre')
        result = model.predict_classes(encoded, verbose=0)
        
        for word, index in t.word_index.items():
            if index == result:
                break
        current_word = current_word + ' ' + word
        
        sentence = sentence + ' ' + word
        
    sentence = init_word + sentence
    
    return sentence

In [76]:
print(sentence_generation(model, t, 'i', 10))

i want to be rich and im not sorry reality tv


In [77]:
print(sentence_generation(model, t, 'how', 10))

how to make facebook more accountable just like in a total
